In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime
import pandas.tseries.offsets as of  # for timeoffset

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

In [2]:
#2.target from csvs
df_bf = pd.read_csv("bf_target_day_using_weekly_v3.csv",encoding='CP949')
df_af = pd.read_csv("af_target_day_using_weekly_v3.csv",encoding='CP949')
del df_bf['Unnamed: 0']
del df_af['Unnamed: 0']

In [3]:
df2_bf = df_bf[['index', 'shcode','month_ma5_score', 'high_month', 
               'for_net_buy_sum', 'com_net_buy_sum','per_net_buy_sum', 
               'week_ma5_score', 'week_ma20_score',
               'week_golden_cross', 'week_dead_cross', 'week_up', 'week_down']].drop_duplicates()

In [4]:
df_bfaf = pd.merge(df_af, df2_bf, how = 'left', on = ['index', 'shcode'])

In [5]:
uplist = query.query_up_distinct()
tmlist = query.query_tm_distinct()

In [6]:
df_bfaf = pd.merge(df_bfaf, uplist[['shcode','upcode', 'upname']], how = 'left', on = ['shcode'])
df_bfaf = pd.merge(df_bfaf, tmlist[['shcode','tmcode', 'tmname']], how = 'left', on = ['shcode'])

In [8]:
'''
df_bfaf[['index', 'shcode', 'shname', 'target_m', 'date', 'close_ratio', 'open_',
        'low', 'high', 'close', 'min_target', 'max_target', 'range_position',
        'volume', 'volume_ratio', 'spot_range', 'inrange','volume_default', 'squeezed',
        'obv', 'upcode', 'upname', 'tmcode', 'tmname',
        'month_ma5_score', 'high_month','for_net_buy_sum', 'com_net_buy_sum', 'per_net_buy_sum',
        'week_ma5_score', 'week_ma20_score', 'week_golden_cross',
        'week_dead_cross', 'week_up', 'week_down', 'for_net_buy', 'com_net_buy', 'per_net_buy',
        'close_ma5', 'close_ma10','close_ma20', 'close_ma60', 'close_ma120']].to_excel("bfaf_target_day_using_weekly_v4.xlsx",encoding = 'ms949')
'''

In [7]:
df_bfaf = df_bfaf[['index', 'shcode', 'shname', 'target_m', 'date', 'close_ratio', 'open_',
                    'low', 'high', 'close', 'min_target', 'max_target', 'range_position',
                    'volume', 'volume_ratio', 'spot_range', 'inrange','volume_default', 'squeezed',
                    'obv', 'upcode', 'upname', 'tmcode', 'tmname',
                    'month_ma5_score', 'high_month','for_net_buy_sum', 'com_net_buy_sum', 'per_net_buy_sum',
                    'week_ma5_score', 'week_ma20_score', 'week_golden_cross',
                    'week_dead_cross', 'week_up', 'week_down', 'for_net_buy', 'com_net_buy', 'per_net_buy',
                    'close_ma5', 'close_ma10','close_ma20', 'close_ma60', 'close_ma120']]

In [55]:
df_bfaf.iloc[9815]

index                        439
shcode                    089010
shname                     켐트로닉스
target_m                  201911
date                    20191115
close_ratio                    0
open_                      14550
low                        14550
high                       18200
close                      17350
min_target                 14700
max_target                 18200
range_position          0.767123
volume                   4001920
volume_ratio                   1
spot_range              0.757143
inrange                     True
volume_default             False
squeezed                   False
obv                  1.51293e+07
upcode                       319
upname                 화       학
tmcode                       393
tmname                   갤럭시 부품주
month_ma5_score                1
high_month                 15450
for_net_buy_sum           967285
com_net_buy_sum            44598
per_net_buy_sum      -1.0113e+06
week_ma5_score                 1
week_ma20_

In [4]:
#uplist[uplist['shcode']=='000080']
#uplist[uplist['shcode']=='000080'].groupby(['shcode']).count()

In [3]:
#tmlist[tmlist['shcode']=='000080']
#tmlist[tmlist['shcode']=='000080'].groupby(['shcode']).count()

In [ ]:
#df_bfaf2 = df_bfaf[['index','shcode']].drop_duplicates()

In [16]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csvs
df = pd.read_csv("high_ratio_0.05_volume_money_20000000000_v4.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
    
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

#5. target day 확인
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")

#(1) for jupo
#interval calculate before 12 month 
init_jupo_timestamp = target_day_list + of.MonthBegin(n= -12)
end_jupo_timestamp = target_day_list
fin_result['init_jupo'] = init_jupo_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_jupo'] = end_jupo_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result = fin_result.reset_index(drop = True)


#fin_result = fin_result[fin_result['target_m']>='201909'].reset_index()
#del fin_result['index']
#fin_result['index'] = fin_result.index

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [18]:
fin_result.head()

,interval_end,shcode,target_day,target_m,shname,init_jupo,end_jupo
0,20190605,000080,20190424,201904,하이트진로,20180501,20190424
1,20190819,000140,20190708,201907,하이트진로홀딩스,20180801,20190708
2,20190909,000140,20190729,201907,하이트진로홀딩스,20180801,20190729
3,20190528,000150,20190416,201904,두산,20180501,20190416
4,20190906,000150,20190726,201907,두산,20180801,20190726


In [19]:
fin_result.shape

(2471, 7)

In [20]:
fin_result.iloc[438:440]

,interval_end,shcode,target_day,target_m,shname,init_jupo,end_jupo
438,20190429,017900,20190318,201903,광전자,20180401,20190318
439,20190424,017940,20190313,201903,E1,20180401,20190313


In [7]:
fin_result.columns

Index(['interval_end', 'shcode', 'target_day', 'target_m', 'shname',
       'init_jupo', 'end_jupo'],
      dtype='object')

In [54]:
# 
a = query.query_finance(fin_result['shcode'].iloc[439])
a['num_total'] = a['total_vol'].astype(int) * 1000

# for jupo query
b = query.query_jupo_vol(fin_result['shcode'].iloc[439], fr=fin_result['init_jupo'].iloc[439], to=fin_result['end_jupo'].iloc[439])

# cumsum index
c = b.groupby(['shcode','date']).sum().cumsum()
# multi index to single index
c = c.reset_index(level=['shcode','date'])

# calculate max volume
c_for_value_ratio = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
                       'jong_vol', 'fund_vol', 'etcom_vol', 'for_reg_vol', 'for_noreg_vol',
                       'nat_no_vol', 'per_vol', 'com_vol', 'for_vol', 'etc_vol']] / a['num_total'].iloc[0]

c['max_vol'] = c_for_value_ratio.max(axis=1)
c['max_jupo'] = c_for_value_ratio.idxmax(axis=1)

c_for_value_ratio = c_for_value_ratio.rename(columns={'samo_vol' : 'samo_vol_ratio', 'sec_vol' : 'sec_vol_ratio', 
                                  'ins_vol' : 'ins_vol_ratio', 'tusin_vol' : 'tusin_vol_ratio', 'bank_vol' : 'bank_vol_ratio',
                                  'jong_vol' : 'jong_vol_ratio', 'fund_vol' : 'fund_vol_ratio', 'etcom_vol' : 'etcom_vol_ratio',
                                  'for_reg_vol' : 'for_reg_vol_ratio', 'for_noreg_vol' : 'for_noreg_vol_ratio',
                                  'nat_no_vol' : 'nat_no_vol_ratio', 'per_vol' : 'per_vol_ratio', 'com_vol' : 'com_vol_ratio',
                                  'for_vol' : 'for_vol_ratio', 'etc_vol' : 'etc_vol_ratio'})

c_final = pd.concat([c, c_for_value_ratio], axis=1) # column bind

#c_for_value_ratio = c_for_value / b['num_total'].iloc[0]
#cc = pd.concat([c[['shcode','date']], c_for_value_ratio], axis=1) # column bind

#c = c.assign(max3_sum=np.sum(np.partition(c_for_value.values, -3)[:, -3:], 1))

In [39]:
b.columns

Index(['date', 'shcode', 'close', 'volume', 'volume_money', 'high_52',
       'high_52_day', 'low_52', 'low_52_day', 'rotate_ratio', 'burn_ratio',
       'per', 'total_vol', 'ev_ratio', 'vol_scale', 'market_value'],
      dtype='object')

In [30]:
b['total_vol'].astype(int) * 1000

0    6860000
Name: total_vol, dtype: int32

In [50]:
c_final.columns

Index(['shcode', 'date', 'samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol',
       'bank_vol', 'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol',
       'for_reg_vol', 'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol',
       'etc_vol', 'max_vol', 'max_jupo', 'samo_vol', 'sec_vol', 'ins_vol',
       'tusin_vol', 'bank_vol', 'jong_vol', 'fund_vol', 'etcom_vol',
       'for_reg_vol', 'for_noreg_vol', 'nat_no_vol', 'per_vol', 'com_vol',
       'for_vol', 'etc_vol'],
      dtype='object')

In [55]:
c_final.tail()

,shcode,date,samo_vol,sec_vol,ins_vol,tusin_vol,bank_vol,jong_vol,fund_vol,etcom_vol,...,jong_vol_ratio,fund_vol_ratio,etcom_vol_ratio,for_reg_vol_ratio,for_noreg_vol_ratio,nat_no_vol_ratio,per_vol_ratio,com_vol_ratio,for_vol_ratio,etc_vol_ratio
225,017940,20190307,2799,15035,-26881,49836,-9818,-11037,18087,-7632,...,-0.001609,0.002637,-0.001113,0.013183,-0.000005,0.000568,-0.018177,0.006111,0.013178,-0.000544
226,017940,20190308,2799,15186,-26718,49888,-9831,-11037,18429,-8833,...,-0.001609,0.002686,-0.001288,0.013144,-0.000005,0.000568,-0.018064,0.006212,0.013139,-0.000719
227,017940,20190311,2881,15385,-26621,49887,-9843,-11037,18950,-8833,...,-0.001609,0.002762,-0.001288,0.013014,-0.000005,0.000568,-0.018062,0.006341,0.013009,-0.000719
228,017940,20190312,2884,11146,-26213,51910,-9876,-11569,19253,-9834,...,-0.001686,0.002807,-0.001434,0.012711,-0.000007,0.000568,-0.017311,0.006040,0.012704,-0.000865
229,017940,20190313,-9564,-8663,-33947,49828,-9583,-11969,23373,-17702,...,-0.001745,0.003407,-0.002580,0.011332,-0.000014,0.000568,-0.009229,0.000492,0.011318,-0.002012


In [41]:
cc.shape

(230, 17)

In [34]:
c.shape

(230, 19)

In [46]:
c.tail()

,shcode,date,samo_vol,sec_vol,ins_vol,tusin_vol,bank_vol,jong_vol,fund_vol,etcom_vol,per_vol,for_reg_vol,for_noreg_vol,nat_no_vol,com_vol,for_vol,etc_vol,max_vol,max_jupo
225,017940,20190307,2799,15035,-26881,49836,-9818,-11037,18087,-7632,-124691,90436,-33,3899,41920,90403,-3733,0.013183,for_reg_vol
226,017940,20190308,2799,15186,-26718,49888,-9831,-11037,18429,-8833,-123918,90169,-33,3899,42615,90136,-4934,0.013144,for_reg_vol
227,017940,20190311,2881,15385,-26621,49887,-9843,-11037,18950,-8833,-123907,89274,-35,3899,43501,89239,-4934,0.013014,for_reg_vol
228,017940,20190312,2884,11146,-26213,51910,-9876,-11569,19253,-9834,-118751,87197,-46,3899,41434,87151,-5935,0.012711,for_reg_vol
229,017940,20190313,-9564,-8663,-33947,49828,-9583,-11969,23373,-17702,-63313,77737,-96,3899,3374,77641,-13803,0.011332,for_reg_vol


In [ ]:
13905000 - 4056934

In [56]:
c_sam2 = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
           'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
           'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']]
c_sam2.sum(axis=1)

0          768
1         1391
2         4713
3         8197
4         5364
5         5952
6         9889
7         3784
8         4281
9         6685
10        5017
11      -12412
12      -20046
13      -21852
14      -31687
15      -28676
16      -36482
17      -40789
18      -44662
19       14222
20       62999
21       61665
22       68318
23       70326
24       60426
25       55683
26       52569
27       40983
28       34815
29       23644
        ...   
216     209352
217     341543
218     271352
219     510918
220     859740
221     884779
222     897740
223     914734
224     947794
225    1002866
226    1002083
227    1030267
228    1133791
229    1090623
230    1146198
231    1153030
232    1296123
233    1334199
234    1313963
235    1468573
236    1475529
237    1455159
238    1392198
239    1440078
240    1506026
241    1443628
242    1416555
243    1398161
244    1392433
245    1403282
Length: 246, dtype: int64

In [46]:
c_sam = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
           'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
           'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']].iloc[245]

In [48]:
c_sam

samo_vol         -757529
sec_vol           -83176
ins_vol           157171
tusin_vol         620899
bank_vol            7840
jong_vol         -152048
fund_vol          293159
etcom_vol           6385
per_vol         -1403282
for_reg_vol      1308754
for_noreg_vol       1827
nat_no_vol             0
com_vol            86316
for_vol          1310581
etc_vol             6385
Name: 245, dtype: int64

In [51]:
c_sam.sum(axis=0)

1403282

In [41]:
# 외인
1310581 / 13905000

0.09425249910104279

In [ ]:
1403282 / 13905000

In [44]:
# 투신
3240234 / 13905000

0.23302653721682848

In [65]:
13905000 - 4056939

9848061

In [67]:
1098523 / 9848061

0.11154713603012817

In [16]:
b = query.query_finance(fin_result['shcode'].iloc[414])

In [17]:
b

,date,shcode,close,volume,volume_money,high_52,high_52_day,low_52,low_52_day,rotate_ratio,burn_ratio,per,total_vol,ev_ratio,vol_scale,market_value
0,20191203,079960,17350,145243,2469,19500,20191119,9000,20190107,15.95,1.85,22.73,7860,40,00001,1348


In [35]:
query.query_finance(fin_result['shcode'].iloc[439])

,date,shcode,close,volume,volume_money,high_52,high_52_day,low_52,low_52_day,rotate_ratio,burn_ratio,per,total_vol,ev_ratio,vol_scale,market_value
0,20191203,089010,16300,440065,7079,18500,20191119,5230,20181226,13.93,3.16,103.49,13905,40,00001,2225


In [23]:
#상장주식수
type(b['total_vol'].iloc[0])

str

In [25]:
c.iloc[235]

shcode             079960
date             20191115
samo_vol            48992
sec_vol             99560
ins_vol              1584
tusin_vol           31333
bank_vol           -90339
jong_vol             1224
fund_vol            -1584
etcom_vol           21943
per_vol            229600
for_reg_vol       -329458
for_noreg_vol      -12855
nat_no_vol              0
com_vol             90770
for_vol           -342313
etc_vol             21943
max_vol            229600
max_jupo          per_vol
max3_sum           419930
Name: 235, dtype: object

In [28]:
419930 / 7860000

0.05342620865139949

In [24]:
b['total_vol'].astype(int) * 1000

0    7860000
Name: total_vol, dtype: int32

In [ ]:
'최고가_52','최고가일_52','최저가_52','최저가일_52','소진율','회전율','PER'
'상장주식수_천','증거금율','수량단위',